In [11]:
pip install numpy


[notice] A new release of pip is available: 25.0 -> 25.1.1
[notice] To update, run: /opt/homebrew/Cellar/jupyterlab/4.4.1_1/libexec/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [12]:
pip install pandas


[notice] A new release of pip is available: 25.0 -> 25.1.1
[notice] To update, run: /opt/homebrew/Cellar/jupyterlab/4.4.1_1/libexec/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [13]:
import numpy as np
import pandas as pd # Optional: For nice table display
from electre_dss import ElectreSolver

In [14]:
# 1. Define the Data

decision_matrix = np.array([
    [80, 70, 80, 70, 90], # HP1
    [80, 80, 70, 70, 90], # HP2
    [90, 70, 80, 70, 80]  # HP3
])

weights = np.array([5, 4, 3, 4, 2])

# C1 (Harga), C4 (Berat) = cost
# C2 (Kamera), C3 (Memori), C5 (Keunikan) = benefit
criteria_types = ['cost', 'benefit', 'benefit', 'cost', 'benefit']

alternative_names = ['HP1', 'HP2', 'HP3']
criteria_names = ['C1(Harga)', 'C2(Kamera)', 'C3(Memori)', 'C4(Berat)', 'C5(Keunikan)']

print("--- Input Data ---")
print("Decision Matrix:")
display(pd.DataFrame(decision_matrix, index=alternative_names, columns=criteria_names))
print(f"\nWeights: {weights}")
print(f"Criteria Types: {criteria_types}")

--- Input Data ---
Decision Matrix:


,C1(Harga),C2(Kamera),C3(Memori),C4(Berat),C5(Keunikan)
HP1,80,70,80,70,90
HP2,80,80,70,70,90
HP3,90,70,80,70,80



Weights: [5 4 3 4 2]
Criteria Types: ['cost', 'benefit', 'benefit', 'cost', 'benefit']


In [15]:
# 2. Initialize and run the solver
print("\n--- Running Solver ---")
solver = ElectreSolver(
    decision_matrix=decision_matrix,
    weights=weights,
    criteria_types=criteria_types,
    alternative_names=alternative_names,
    criteria_names=criteria_names
)

solver.solve() # This performs all calculations


--- Running Solver ---


In [16]:
# 3. Get and print the final ranking
ranking = solver.get_ranking()
print("\n------ FINAL RANKING ------")
for i, name in enumerate(ranking):
    print(f"{i+1}. {name}")


------ FINAL RANKING ------
1. HP2
2. HP1
3. HP3


In [17]:
# 4. Optional: Display intermediate results for verification
print("\n------ INTERMEDIATE RESULTS (Verification) ------")
results = solver.get_intermediate_results()

print(f"\nConcordance Threshold (c): {results['concordance_threshold']:.3f}")
print("Concordance Matrix (C):")
display(pd.DataFrame(results['concordance_matrix'], index=alternative_names, columns=alternative_names).round(3))

print(f"\nDiscordance Threshold (d): {results['discordance_threshold']:.4f}")
print("Discordance Matrix (D):")
display(pd.DataFrame(results['discordance_matrix'], index=alternative_names, columns=alternative_names).round(3))

print("\nAggregate Dominance Matrix (E):")
display(pd.DataFrame(results['aggregate_dominance_matrix'], index=alternative_names, columns=alternative_names))


------ INTERMEDIATE RESULTS (Verification) ------

Concordance Threshold (c): 13.333
Concordance Matrix (C):


,HP1,HP2,HP3
HP1,0.0,14.0,18.0
HP2,15.0,0.0,15.0
HP3,11.0,7.0,0.0



Discordance Threshold (d): 0.7021
Discordance Matrix (D):


,HP1,HP2,HP3
HP1,0.000,0.909,0.000
HP2,0.652,0.000,0.652
HP3,1.000,1.000,0.000



Aggregate Dominance Matrix (E):


,HP1,HP2,HP3
HP1,0,0,1
HP2,1,0,1
HP3,0,0,0
